In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Cargar datos de usuarios y deportes
datos_usuario = pd.read_csv("../Raw_Datasets/Datos_Ficticios.csv")
deportes = pd.read_csv("deportes.csv")

# Preprocesamiento de datos de usuarios
X_usuario = datos_usuario[["edad", "estatura", "peso", "nivel_actividad", "consumo_calorico"]]
y_usuario = datos_usuario["deporte_preferido"]  # Si tienes datos sobre el deporte preferido del usuario

# Crear modelo de regresión logística
modelo = LogisticRegression()
modelo.fit(X_usuario, y_usuario)

# Obtener el perfil del usuario (puedes obtener estos valores a través de un formulario en la app)
edad_usuario = 30
estatura_usuario = 170
peso_usuario = 70
nivel_actividad_usuario = 3  # 1: Bajo, 2: Moderado, 3: Alto
consumo_calorico_usuario = 250

# Predecir el deporte recomendado para el usuario
perfil_usuario = [[edad_usuario, estatura_usuario, peso_usuario, nivel_actividad_usuario, consumo_calorico_usuario]]
deporte_recomendado = modelo.predict(perfil_usuario)[0]

# Filtrar los deportes recomendados para el usuario según su nivel de actividad
deportes_recomendados_nivel1 = deportes[deportes["intensidad"] <= nivel_actividad_usuario]

# Filtrar el deporte específico recomendado por el modelo
deporte_recomendado_nivel1 = deportes_recomendados_nivel1[deportes_recomendados_nivel1["tipo_deporte"] == deporte_recomendado]

# Imprimir el resultado
print(f"Deporte recomendado para el usuario: {deporte_recomendado_nivel1['nombre_deporte'].values[0]}")
